In [63]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import gradio as gr

In [64]:
# Define image dimensions and batch size
img_width, img_height = 128, 128
batch_size = 32

In [65]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,   # Normalize pixel values to the range [0, 1]
    shear_range=0.2,     # Shear transformations
    zoom_range=0.2,      # Random zoom
    horizontal_flip=True ,# Horizontal flipping
    validation_split=0.2 
)

In [66]:
# Load and augment training data
train_data_generator = train_datagen.flow_from_directory(
    'path to dataset',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary' ,# For binary classification
    subset='training'
)

Found 1216 images belonging to 2 classes.


In [67]:
# Load validation data with the same generator
test_data_generator = train_datagen.flow_from_directory(
    'path to dataset',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',  # For binary classification
    subset='validation'  # Specify the validation split
)

Found 304 images belonging to 2 classes.


In [68]:
# Create the CNN model
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification


In [69]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


In [70]:
# Train the model with validation data
epochs = 20
history = model.fit(
    train_data_generator,
    epochs=epochs,
    steps_per_epoch=train_data_generator.samples // batch_size,
    validation_data=test_data_generator,
    validation_steps=test_data_generator.samples // batch_size,
    verbose=1
)

Epoch 1/20
38/38 [==============================] - 76s 2s/step - loss: 0.4340 - accuracy: 0.8388 - val_loss: 0.2140 - val_accuracy: 0.9236
Epoch 2/20
38/38 [==============================] - 72s 2s/step - loss: 0.2427 - accuracy: 0.8972 - val_loss: 0.1853 - val_accuracy: 0.9340
Epoch 3/20
38/38 [==============================] - 78s 2s/step - loss: 0.1785 - accuracy: 0.9334 - val_loss: 0.1292 - val_accuracy: 0.9722
Epoch 4/20
38/38 [==============================] - 74s 2s/step - loss: 0.1600 - accuracy: 0.9334 - val_loss: 0.1009 - val_accuracy: 0.9722
Epoch 5/20
38/38 [==============================] - 75s 2s/step - loss: 0.1525 - accuracy: 0.9457 - val_loss: 0.1385 - val_accuracy: 0.9722
Epoch 6/20
38/38 [==============================] - 88s 2s/step - loss: 0.1379 - accuracy: 0.9490 - val_loss: 0.1068 - val_accuracy: 0.9757
Epoch 7/20
38/38 [==============================] - 70s 2s/step - loss: 0.1250 - accuracy: 0.9613 - val_loss: 0.1020 - val_accuracy: 0.9792
Epoch 8/20
38/38 [==

In [71]:
import matplotlib.pyplot as plt
plt.switch_backend('tkagg')  

In [72]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig('accuracy_plot.png')  # Save accuracy plot as an image file

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.savefig('/loss_plot.png')  # Save loss plot as an image file

plt.tight_layout()

plt.show()  # Display the plots using the default image viewer

In [89]:
from PIL import Image
import warnings

# Suppress all warnings (not recommended unless you're sure)
warnings.filterwarnings("ignore")


In [90]:
# Define a function to make predictions on an input image
def predict_image(input_image):
    # Preprocess the image
    img = tf.image.resize(input_image, (img_width, img_height))
    img = tf.expand_dims(img, axis=0)
    img = img / 255.0  # Normalize pixel values to [0, 1]

    # Make the prediction
    prediction = model.predict(img)
    class_names = ["Fire", "No Fire"]
    return class_names[int(round(prediction[0][0]))]


In [ ]:
# Create a Gradio interface
iface = gr.Interface(
    fn=predict_image,
    inputs="image",
    outputs="text",
    live=True,
    interpretation="default",
      theme="huggingface",  # Set the theme to "huggingface"
    layout="vertical",  # Set the layout to "vertical"
    capture_session=True,  # Enable screenshot capture
    allow_flagging=False,  # Disable flagging option
    title="Forest Fire Predictor",
    description="Upload an image to predict forest fire presence or absence.",
    server_name="Custom Gradio Server",
    server_port=5000,
    server_host="0.0.0.0"
    
)

# Launch the interface
iface.launch(debug=True,share=True)